In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [74]:
repo = 'ada-2024-project-adarable/src'

In [75]:
cd {repo}

/content/drive/MyDrive/ada-2024-project-adarable/src


In [26]:
!pip install requests beautifulsoup4 selenium webdriver_manager

In [27]:
# Install the latest version of Google Chrome
!apt-get update
!apt-get install -y wget curl unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,223 B]
Fetched 260 kB in 2s (158 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/S

In [28]:
# Install ChromeDriver using webdriver_manager
!pip install -q webdriver_manager

In [29]:
# Automatically reload all modules before executing code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
import pandas as pd
movies = pd.read_csv('../data/data_to_scrape/final_scrape_sample1.csv')

In [77]:
movies.shape[0]

3452

In [78]:
movies.head(1)

,Unnamed: 0.1,Unnamed: 0,wikipedia_movie_id,freebase_movie_id,movie_name,movie_release_date,movie_box_office_revenue,movie_runtime,movie_languages,movie_countries,movie_genres,plot_summary
0,0,0,975900,/m/03vyhn,Ghosts of Mars,2001.0,14010832.0,98.0,['English Language'],['United States of America'],"['Thriller', 'Science Fiction', 'Horror', 'Adv...","Set in the second half of the 22nd century, th..."


In [54]:
movies.dtypes

,0
Unnamed: 0.1,int64
Unnamed: 0,int64
wikipedia_movie_id,int64
freebase_movie_id,object
movie_name,object
movie_release_date,float64
movie_box_office_revenue,float64
movie_runtime,float64
movie_languages,object
movie_countries,object


In [79]:
import csv
import os
from web_scraper import WebScraper

# Initialize the scraper object
scraper = WebScraper()

movies_id = movies['wikipedia_movie_id']

# Function to get the checkpoint from a previous run
def get_checkpoint():
    if os.path.exists('checkpoint.txt'):
        with open('checkpoint.txt', 'r') as f:
            last_processed_movie_id = f.read().strip()
            return last_processed_movie_id
    return None

# Function to update the checkpoint after processing a movie
def update_checkpoint(movie_id):
    with open('checkpoint.txt', 'w') as f:
        f.write(str(movie_id))

def scrape_and_save_movie_data(movies):
    # Define CSV file path
    csv_file_path = "../data/imdb_with_currency_final_scrape_sample1.csv"

    # Column names for the CSV file
    fieldnames = [
        "wikipedia_movie_id", "movie_box_office_revenue", "currency_revenue", "budget",
        "currency_budget", "opening_weekend", "currency_opening_weekend", "rating_score", "producer", "release_year"
    ]

    checkpoint = get_checkpoint()  # Get the checkpoint from the last run
    if checkpoint:
        print(f"Resuming from movie ID {checkpoint}")
    else:
        print("Starting from the beginning.")

    # Start from the movie after the checkpoint
    start_index = 0
    if checkpoint:
        # Get the index of the last processed movie
        try:
            start_index = movies[movies['wikipedia_movie_id'] == int(checkpoint)].index[0] + 1
        except IndexError:
            print("Checkpoint movie not found, starting from the first movie.")
            start_index = 0

    # Check if the file exists and open the file for appending
    file_exists = os.path.exists(csv_file_path)
    print(f"File exists: {file_exists}")  # Debug: Check if the file exists

    with open(csv_file_path, mode="a", newline='', encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # If the file is being created for the first time (not exists or empty), write the header
        if not file_exists or os.path.getsize(csv_file_path) == 0:
            print("Writing header to file...")  # Debug: Writing header for first time
            writer.writeheader()

        # Loop through the movies to scrape and save data starting from the checkpoint
        for movie_id in movies_id[start_index:]:
            print(f"Scraping data for movie ID {movie_id}...")  # Debug: Check current movie being processed

            # Scrape the movie info using the ImdbScraper
            movie_info = scraper.get_imdb_infos(movie_id)

            # Check if data is available and write it to the CSV file
            if movie_info:
                print(f"Writing data for movie ID {movie_id}...")  # Debug: Data being written
                writer.writerow(movie_info)
                update_checkpoint(movie_id)  # Update the checkpoint after processing each movie
            else:
                print(f"Failed to scrape data for movie ID {movie_id}")  # Debug: Failed to scrape data

    print(f"Movie data saved to {csv_file_path}.")
    scraper.close()

scrape_and_save_movie_data(movies)

Starting from the beginning.
File exists: False
Writing header to file...
Scraping data for movie ID 975900...
Writing data for movie ID 975900...
Scraping data for movie ID 171005...


KeyboardInterrupt: 

In [ ]:
df = pd.read_csv('../data/imdb_additional_movies_for_sven.csv')

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape[0]

1164

In [ ]:
cleaned_file_path = '../data/imdb_additional_movies_data_half_1_cleaned.csv'
df.to_csv(cleaned_file_path, index=False)

In [ ]:
df_cleaned = pd.read_csv('../data/imdb_additional_movies_data_half_1_cleaned.csv')

In [ ]:
df_cleaned.shape[0]

1240